In [4]:
import mwu_measures
import pandas as pd
import numpy as np
import mwu_measures.processing_corpus
from mwu_measures.corpus_helper import Fetcher

mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
this_corpus = mwu_measures.processing_corpus.process_corpus('bnc', 'small_corpus.txt', chunk_size=10000000, verbose=True)
ngram_selection = mwu_examples['Expression'].tolist()
ngram_selection = [ngram.split() for ngram in ngram_selection]
# ngram_chunks = np.array_split(ngram_selection, 100)

90357 lines processed
178289 lines processed
287795 lines processed
301046 lines processed
we bring the boom


In [5]:
helper = Fetcher(this_corpus.corpus_conn)

In [6]:
helper.conn.execute("SELECT * FROM trigram_db LIMIT 100").fetch_df()

,corpus,ug_1,ug_2,ug_3,big_1,freq
0,A,$1.70,he,reckons,$1.70 he,1.0
1,A,$1.80,on,a,$1.80 on,1.0
2,A,$1.80,or,as,$1.80 or,1.0
3,A,$17.5m,american,express,$17.5m american,1.0
4,A,$2,a,time,$2 a,1.0
...,...,...,...,...,...,...
95,A,(,laugh,but,( laugh,1.0
96,A,(,lionel,jospin,( lionel,1.0
97,A,(,lone,snigger,( lone,1.0
98,A,(,look,at,( look,1.0


In [3]:
# x = helper.fetch_bigrams(ngram_selection, 'bw') # BIgRAM IS OK

In [8]:
trigrams = [[' '.join(ngram), ' '.join([ngram[0], ngram[1]])] for ngram in ngram_selection if len(ngram) == 3]
# trigrams = [[' '.join(ngram), ngram[2]] for ngram in ngram_selection if len(ngram) == 3]

In [10]:
trigrams = pd.DataFrame(trigrams, columns = ['query', 'big_1'])
# trigrams = pd.DataFrame(trigrams, columns = ['query', 'ug_3'])

In [11]:
trigrams

,query,big_1
0,‘tis the season,‘tis the
1,8 ball pool,8 ball
2,a bad break,a bad
3,a big ask,a big
4,a big deal,a big
...,...,...
9722,zest for life,zest for
9723,zip one's lip,zip one's
9724,zip your lip,zip your
9725,zone in on,zone in


In [12]:
this_corpus.corpus_conn.execute(
    """
    CREATE OR REPLACE TEMPORARY TABLE this_query (query TEXT, big_1 TEXT, hash_index UINT64)
""")
this_corpus.corpus_conn.execute(f"""
    INSERT INTO this_query SELECT *, hash(big_1) AS hash_index FROM trigrams
""")

In [12]:
this_corpus.corpus_conn.execute(
    """
    CREATE OR REPLACE TEMPORARY TABLE this_query (query TEXT, ug_3 TEXT, hash_index UINT64)
""")
this_corpus.corpus_conn.execute(f"""
    INSERT INTO this_query SELECT *, hash(ug_3) AS hash_index FROM trigrams
""")

In [13]:
this_corpus.corpus_conn.execute("SELECT * FROM this_query").fetch_df()

,query,big_1,hash_index
0,‘tis the season,‘tis the,367049523107210819
1,8 ball pool,8 ball,13478181352906029041
2,a bad break,a bad,3783471393295332674
3,a big ask,a big,3347211066445533376
4,a big deal,a big,3347211066445533376
...,...,...,...
9722,zest for life,zest for,12779792370419524593
9723,zip one's lip,zip one's,1131825406419085369
9724,zip your lip,zip your,1021777779954715733
9725,zone in on,zone in,9675242506793349944


In [16]:
this_corpus.corpus_conn.execute(
    f"""
    CREATE OR REPLACE TABLE filtered_db AS
    SELECT
        corpus,
        hash(big_1) as hash_index,
        ug_3,
        freq
    FROM trigram_db
    WHERE 
        hash_index IN (
            SELECT hash_index FROM this_query
        )
""")
# this_corpus.corpus_conn.execute(
#     f"""
#     CREATE OR REPLACE TABLE filtered_db AS
#     SELECT
#         corpus,
#         hash(ug_3) as hash_index,
#         ug_1,
#         ug_2,
#         freq
#     FROM trigram_db
#     WHERE 
#         hash_index IN (
#             SELECT hash_index FROM this_query
#         )
# """)

In [17]:
this_corpus.corpus_conn.execute("SELECT * FROM filtered_db").fetch_df()

,corpus,hash_index,ug_1,ug_2,freq
0,A,14322568082006533075,wasps,nest,1.0
1,A,16295433537293246348,wastage,during,1.0
2,A,16330219647563866030,wastage,of,1.0
3,A,5350958569187059494,waste,any,1.0
4,A,2393033049016590498,waste,being,1.0
...,...,...,...,...,...
1789563,H,5137008091097008025,times,not,1.0
1789564,H,4437588317420669275,times,of,1.0
1789565,H,7341120841597100533,times,of,1.0
1789566,H,8802668120057561073,times,of,1.0


In [71]:
# this_corpus.corpus_conn.execute("SELECT * FROM filtered_db WHERE hash_1 = hash('a') AND hash_2 = hash('number') AND ug_3 = 'of'").fetch_df()

,corpus,hash_1,hash_2,ug_3,freq
0,D,584566915021039271,180176074552245289,of,1.0
1,E,584566915021039271,180176074552245289,of,76.0
2,A,584566915021039271,180176074552245289,of,91.0
3,G,584566915021039271,180176074552245289,of,71.0
4,B,584566915021039271,180176074552245289,of,58.0
5,F,584566915021039271,180176074552245289,of,89.0
6,J,584566915021039271,180176074552245289,of,35.0
7,C,584566915021039271,180176074552245289,of,115.0
8,H,584566915021039271,180176074552245289,of,199.0
9,K,584566915021039271,180176074552245289,of,53.0


In [18]:
# this_corpus.corpus_conn.execute(
#     f"""
#     CREATE OR REPLACE TABLE joined_db AS                  
#     SELECT 
#         query,
#         corpus,
#         this_query.ug_1,
#         this_query.ug_2,
#         filtered_db.ug_3,
#         freq
#     FROM this_query
#     INNER JOIN filtered_db
#     ON filtered_db.hash_1 = this_query.hash_1 AND filtered_db.hash_2 = this_query.hash_2
# """)

this_corpus.corpus_conn.execute(
    f"""
    CREATE OR REPLACE TABLE joined_db AS                  
    SELECT 
        query,
        corpus,
        filtered_db.ug_1,
        filtered_db.ug_2,
        this_query.ug_3,
        freq
    FROM this_query
    INNER JOIN filtered_db
    USING(hash_index)
""")

In [22]:
this_corpus.corpus_conn.execute("SELECT * FROM joined_db ORDER BY freq DESC LIMIT 100").fetch_df()

,query,corpus,ug_1,ug_2,ug_3,freq
0,you never know,K,i,don't,know,252.0
1,wouldn't you know,K,i,don't,know,252.0
2,let someone know,K,i,don't,know,252.0
3,in the know,K,i,don't,know,252.0
4,i don't know,K,i,don't,know,252.0
...,...,...,...,...,...,...
95,the majority of,H,the,end,of,231.0
96,the likes of,H,the,end,of,231.0
97,the lack of,H,the,end,of,231.0
98,the bulk of,H,the,end,of,231.0
